# ◆[Backtest](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Strategy.buy) with OANDA API
###### Create Date：2020/02/08　Author：M.Hasegawa
### ────────────────────────────────────────────────────────────────

#### 【table of contents】
- 0.[**Import module**](#Import_module)
- 1.[**Import data**](#Import_data)
- 2.[**Strategy**](#Strategy)
- 3.[**Backtest**](#Backtest)

- ref:https://kernc.github.io/backtesting.py/doc/examples/Multiple%20Time%20Frames.html
- ref:https://saidataisei.hatenablog.com/entry/2019/10/13/003622
- ref:http://mmorley.hatenablog.com/entry/fx_backtesting01

## 0. Import module<a id='Import_module'></a>
- pip install backtesting
- pip install mpl_finance
- pip install oandapyV20
- pip install git+https://github.com/oanda/oandapy.git
- pip install TA_Lib-0.4.17-cp37-cp37m-win_amd64.whl

###### TA-LIB
- ref: https://qiita.com/ConnieWild/items/cb50f36425a683c914d2
- ref: https://www.lfd.uci.edu/~gohlke/pythonlibs/#ta-lib

In [1]:
%matplotlib inline
import oandapy
import pytz
import configparser
import pandas as pd
import numpy  as np
import matplotlib.dates as mdates
import oandapyV20.endpoints.instruments as instruments
import talib as ta

from datetime         import datetime
from oandapyV20       import API
from backtesting      import Backtest, Strategy
from backtesting.lib  import crossover
from backtesting.lib  import resample_apply
from backtesting.test import SMA, GOOG
from pandas.core import resample

config = configparser.ConfigParser()
config.read('./config_OANDA.txt')
account_id = config['oanda']['account_id']
api_key = config['oanda']['api_key']

import warnings
warnings.filterwarnings("ignore")

# ============================================================================
# Conv Japan Time
# ============================================================================
def iso_to_jp(iso):
    date = None
    try:
        date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%fZ')
        date = pytz.utc.localize(date).astimezone(pytz.timezone("Asia/Tokyo"))
    except ValueError:
        try:
            date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%f%z')
            date = date.astimezone(pytz.timezone("Asia/Tokyo"))
        except ValueError:
            pass
    return date

# ============================================================================
# Conv Format
# ============================================================================
def date_to_str(date):
    if date is None:
        return ''
    return date.strftime('%Y/%m/%d %H:%M:%S')

# ============================================================================
# SMA
# ============================================================================
def SMA(array, n):
    return pd.Series(array).rolling(n).mean()

# ============================================================================
# RSI
# ============================================================================
def RSI(array, n):
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

# ============================================================================
# RCI
# ============================================================================
def RCI(close, period):
    rank_period = np.arange(period, 0, -1)
    length = len(close)
    rci = np.zeros(length)
    
    for i in range(length):
        if i < period - 1:
            rci[i] = 0
        else :
            rank_price = pd.Series(close)[i - period + 1: i + 1].rank(method='min', ascending = False).values
            rci[i] = np.int32((1 - (6 * sum((rank_period - rank_price)**2)) / (period**3 - period)) * 100)
            
    return rci

# ============================================================================
# BB
# ============================================================================
def bb(array, n):
    gain=pd.DataFrame(array)
    gain.columns=['close']
    upper2, middle, lower2 = ta.BBANDS(gain.close, n,2,2,0)
    gain['bb_upper'] = upper2
    gain['bb_lower'] = lower2
    return gain['bb_upper'],gain['bb_lower']

# ============================================================================
# ADX
# ============================================================================
def adx(array,n):
    gain=pd.DataFrame(array)
    gain=gain.T
    gain.columns=['close','high','low']
    gain['adx'] = ta.ADX(gain['high'],gain['low'],gain['close'],n)
    return gain['adx']

C:\Users\arsma\Anaconda3\lib\site-packages\backtesting\_plotting.py:37: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# 1. Import data<a id='Import_data'></a>

URL:http://developer.oanda.com/rest-live/rates/

- M30・・・30 minute candlesticks, hour alignment
- H1・・・1 hour candlesticks, hour alignment
- H4・・・4 hour candlesticks, day alignment 約1年間：count=1560
- D・・・1 day candlesticks, day alignment
- W・・・1 week candlesticks, aligned to start of week
- M・・・1 month candlesticks, aligned to first day of the month

In [2]:
oa = oandapy.API(environment="live", access_token=api_key)
data = oa.get_history(instrument="USD_JPY", granularity='H4', count=1560)

df = pd.DataFrame(data["candles"])

df['time'] = df['time'].apply(lambda x: iso_to_jp(x))   # 日本時間に変換
df['time'] = df['time'].apply(lambda x: date_to_str(x)) # 形式変換（文字列型）
df["time"] = pd.to_datetime(df["time"])                  # 型変換
#df["time"] = df["time"].apply(mdates.date2num)          # 数値変換

df_ohlc = df[["time","openAsk","highAsk","lowAsk","closeAsk"]].copy()
df_ohlc = df_ohlc.rename(columns={"openAsk":"Open","highAsk":"High","lowAsk":"Low","closeAsk":"Close"})
df_ohlc = df_ohlc.set_index("time")

print('df_ohlc.shape=',df_ohlc.shape)
display(df_ohlc.head(2))
display(df_ohlc.tail(2))

df_ohlc.shape= (1560, 4)


,Open,High,Low,Close
time,,,,
2019-02-11 11:00:00,109.811,110.024,109.798,109.990
2019-02-11 15:00:00,109.991,110.289,109.949,110.258


,Open,High,Low,Close
time,,,,
2020-02-10 11:00:00,109.793,109.885,109.735,109.783
2020-02-10 15:00:00,109.781,109.859,109.766,109.840


# 2. Strategy<a id='Strategy'></a>

## 2-1. SmaCross

In [3]:
class SmaCross(Strategy):
    n1 = 5
    n2 = 21
 
    def init(self): # ヒストリカルデータの行ごとに呼び出される（データの2行目から開始）
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        if crossover(self.sma1, self.sma2): # 短期が長期を超えたらゴールデンクロスで買い注文
            self.buy() # 現在のポジションを閉じて、所持金分買う
        elif crossover(self.sma2, self.sma1): # 短期が長期を下回ったらデッドクロスで売り注文
            self.sell() # 現在のポジションを閉じて、所持金分売る

## 2-2. RSIStrategy

In [4]:
class RSIStrategy(Strategy):
    d_rsi = 30
    level = 70
    
    def init(self):
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
            
    def next(self):
        price = self.data.Close[-1]
        
        # まだポジションがなく、すべての条件が満たす場合
        if (not self.position 
            and self.daily_rsi[-1] > self.level 
            and self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] 
            and price > self.ma10[-1]):
            
            # 次回のオープン時に市場価格で購入しますが、8％の固定ストップロスを設定
            self.buy(sl=.92 * price)
            
        # 価格が10日移動平均より2％以上下がった場合
        elif price < .98 * self.ma10[-1]:
            self.position.close()


## 2-3. RCIStrategy

#### H4×1000
- Duration= 234 days 21:00:00
- Equity Final($)= 1146.2
- Trades= 20
- Win Rate(%)= 95.0
- Return(%)= 14.6
- Avg. Trade Duration= 9 days 04:00:00
- SQN= 4.8

#### H4×1560（1年）
- Duration= 364 days 04:00:00
- Equity Final($)= 1184.1
- Trades= 31
- Win Rate(%)= 90.3
- Return(%)= 18.4
- Avg. Trade Duration= 9 days 16:00:00
- SQN= 4.0

In [5]:
class RCIStrategy(Strategy):
    n1 = 21
    n2 = 75
    
    rci_short = 9
    rci_middle = 21
    rci_long = 52
    
    def init(self):
        self.rci_short = self.I(RCI, self.data.Close, self.rci_short)
        self.rci_middle = self.I(RCI, self.data.Close, self.rci_middle)
        self.rci_long = self.I(RCI, self.data.Close, self.rci_long)
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        
    def next(self):
        price = self.data.Close[-1]        
        
        # ▼ロング
        # まだポジションがなく、すべての条件が満たす場合
        if (not self.position and 
            ((1==0) 
             # ▼ロングのEntryルール
             or (self.rci_short[-1] < -70 and self.rci_long[-1] < -60)
             or (self.rci_short[-1] < -70 and self.rci_middle[-1] < -60)
            )):
            
            self.buy(sl=.98 * price) # 次回オープン時に市場価格で購入、2％の固定ストップロスを設定
            
        elif (self.position.is_long and 
              ((1==0) 
               # ▼ロングのExitルール
               or (self.rci_middle[-1] > 88) 
               or (self.rci_long[-1] > 55)
               or (self.rci_short[-1] > 80 and self.rci_long[-1] > 30) # 20～30
              )):
            
            # クローズ
            self.position.close() 
            
        # ▼ショート
        # まだポジションがなく、すべての条件が満たす場合
        if (not self.position and 
            ((1==0) 
             # ▼ショートのEntryルール
             or (self.rci_short[-1] > 70 and self.rci_long[-1] > 60)
             or (self.rci_short[-1] > 70 and self.rci_middle[-1] > 60)
            )
           ):
            
            self.sell(sl=1.02 * price) # 次回オープン時に市場価格で購入、2％の固定ストップロスを設定
            
        elif (self.position.is_short and 
              ((1==0) 
               # ▼ショートのExitルール
               or (self.rci_middle[-1] < -85) 
               or (self.rci_long[-1] < -55)
               or (self.rci_short[-1] < -75 and self.rci_middle[-1] < -75)
              )):
            
            # クローズ
            self.position.close() 

## 2-4. bb_band

In [6]:
class bb_band(Strategy):
    n = 25
    n2 = 20
    diff1 = 30
    adx_h = 20
    
    def init(self):
        self.bb_upper2,self.bb_lower2 = self.I(bb,self.data.Close,self.n)
        self.adx=self.I(adx,[self.data.Close,self.data.High,self.data.Low],self.n2)

    def next(self):
        
        price = self.data.Close[-1]
        diff=0.01*self.diff1
        
        #ポジション確認（long:1,nun:0,short:-1）
        position=0
        if(self.position):
            # ポジションあり
            if(self.position.is_long):
                # ロング
                position=1
                if(self.position.open_price+diff<self.data.Close):
                    self.position.close
                    position=0
            elif(self.position.is_short):
                # ショート
                position=-1
                if(self.position.open_price-diff>self.data.Close):
                    self.position.close
                    position=0
                    
        #レンジ相場であることを確認
        if(self.adx < self.adx_h):
            #+σ2より大きいとき,売り
            if(self.data.Close>self.bb_upper2):
                if(position==0):
                    self.sell(sl=1.02 * price)
                if(position==1):
                    self.position.close
                    self.sell(sl=1.02 * price)
            #-σ2より小さいとき、買い
            elif(self.data.Close<self.bb_lower2):
                if(position==0):
                    self.buy(sl=.98 * price)
                if(position==-1):
                    self.position.close
                    self.buy(sl=.98 * price)

In [7]:
class RCIStrategy2(Strategy):
    n1 = 21
    n2 = 75
    
    rci_short = 9
    rci_middle = 21
    rci_long = 52
    
    def init(self):
        self.rci_short = self.I(RCI, self.data.Close, self.rci_short)
        self.rci_middle = self.I(RCI, self.data.Close, self.rci_middle)
        self.rci_long = self.I(RCI, self.data.Close, self.rci_long)
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        
    def next(self):
        price = self.data.Close[-1]        
        
        # ▼ロング
        # まだポジションがなく、すべての条件が満たす場合
        if (not self.position and 
            ((1==0) 
             # ▼ロングのEntryルール
             or (self.rci_short[-1] < -70 and self.rci_long[-1] < -60)
             or (self.rci_short[-1] < -70 and self.rci_middle[-1] < -60)
            )):
            
            self.buy(sl=.98 * price) # 次回オープン時に市場価格で購入、2％の固定ストップロスを設定
            
        elif (self.position.is_long and 
              ((1==0) 
               # ▼ロングのExitルール
               or (self.rci_middle[-1] > 88) 
               or (self.rci_long[-1] > 55)
               or (self.rci_short[-1] > 80 and self.rci_long[-1] > 30) # 20～30
              )):
            
            # クローズ
            self.position.close() 
            
        # ▼ショート
        # まだポジションがなく、すべての条件が満たす場合
        if (not self.position and 
            ((1==0) 
             # ▼ショートのEntryルール
             or (self.rci_short[-1] > 70 and self.rci_long[-1] > 60)
             or (self.rci_short[-1] > 70 and self.rci_middle[-1] > 60)
            )
           ):
            
            self.sell(sl=1.02 * price) # 次回オープン時に市場価格で購入、2％の固定ストップロスを設定
            
        elif (self.position.is_short and 
              ((1==0) 
               # ▼ショートのExitルール
               or (self.rci_middle[-1] < -85) 
               or (self.rci_long[-1] < -55)
               or (self.rci_short[-1] < -75 and self.rci_middle[-1] < -75)
              )):
            
            # クローズ
            self.position.close() 

# 3. Backtest<a id='Backtest'></a>

- URL:https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Strategy.buy

In [8]:
# SmaCross RSIStrategy RCIStrategy bb_band
bt = Backtest(df_ohlc,
              RCIStrategy2,
              cash=1000,  # 所持金1000ドル(=約10万)
              commission=0.0002, # 取引手数料（為替価格に対する倍率で指定、為替価格100円でcommission=0.0005なら0.05円）
              trade_on_close=False
             )
out=bt.run()
#print(out)
print('Duration=',out['Duration'])
print('Equity Final($)=',round(out['Equity Final [$]'],1))
print('Trades=',out['# Trades'])
print('Win Rate(%)=',round(out['Win Rate [%]'],1))
print('Return(%)=',round(out['Return [%]'],1))
print('Avg. Trade Duration=',out['Avg. Trade Duration'])
print('SQN=',round(out['SQN'],1))

bt.plot()

Duration= 364 days 04:00:00
Equity Final($)= 1183.6
Trades= 31
Win Rate(%)= 90.3
Return(%)= 18.4
Avg. Trade Duration= 9 days 16:00:00
SQN= 4.0


- Start・・・・・・・・・・・ヒストリカルデータの開始日時
- End・・・・・・・・・・・・ヒストリカルデータの終了日時
- Duration・・・・・・・・・ ヒストリカルデータの期間
- Exposure (%) ・・・・・・・ポジションを持っていた期間の割合（ポジションを持っていた期間÷全期間×100）
- Equity Final・・・・・・・ 最終金額
- Equity Peak・・・・・・・・最高金額
- Return (%)・・・・・・・・ 利益率=損益÷開始時所持金×100
- Buy & Hold Return (%)・・・（（終了時の終値 - 開始時の終値）÷ 開始時の終値）の絶対値×100
- Max. Drawdown (%)・・・・・最大下落率
- Avg. Drawdown (%)・・・・・平均下落率
- Max. Drawdown Duration・・ 最大下落期間
- Avg. Drawdown Duration・・ 平均下落期間
- Trades・・・・・・・・・・ 取引回数
- Win Rate (%)・・・・・・・ 勝率=勝ち取引回数÷全取引回数×100
- Best Trade (%)・・・・・・ 1回の取引での利益の最大値÷所持金×100
- Worst Trade (%)・・・・・・1回の取引での損失の最大値÷所持金×100
- Avg. Trade (%)・・・・・・ 損益の平均値÷所持金×100
- Max. Trade Duration・・・・1回の取引での最長期間
- Avg. Trade Duration・・・・1回の取引での平均期間
- Expectancy (%)・・・・・・ 期待値=平均利益×勝率＋平均損失×敗率（１取引で期待できる利益、正は資産が増え、負は資産が減る。）
- SQN(取引システム評価)・・・1.6～1.9：平均以下、2.0～2.4：平均、2.5～2.9：良い、3.0～5.0：素晴らしい、5.1～6.9：最高、7.0～：聖杯？　※取引数が30以上の場合、SQN値は信頼できるとみなす。
- Sharpe Ratio・・・・・・・標準偏差に対するリターンの比率 ※シャープレシオとは利益とリスクの比率のことで、値が大きいほど資産曲線がなめらかになり安定性のある利益が見込めます。
- Sortino Ratio・・・・・・・下方リスクに対するリターンの比率 ※シャープレシオだけでは分からない下方リスクの抑制度合いを判断する場合に使われます。通常、この数値が大きいほど優れている（下落局面に強い）ことを示します。
- Calmar Ratio・・・・・・・最大損失率に対する年間平均収益の比率 ※値が低いほど指定された期間に渡ってリスク調整ベースで実行された投資は悪化し、値が高いほどパフォーマンスが向上します。

- 最上部のEquityのグラフは資金推移を表します。
- 2段目のProfit / lossはトレードロジックで行われたトレードを利益と損益で可視化します。
- 最下部は実際の終値のレートと単純移動平均（短期と長期）、さらにトレードも併せて可視化します。

In [9]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 5000)

print(out)

display(out._trade_data)
#display(out._trade_data[['Equity','Exit Entry','Exit Position','Entry Price','Exit Price','Returns']])
out._trade_data.to_csv('./Backtest_trade_data.csv')

Start                     2019-02-11 11:00:00
End                       2020-02-10 15:00:00
Duration                    364 days 04:00:00
Exposure [%]                          82.1968
Equity Final [$]                      1183.61
Equity Peak [$]                       1184.24
Return [%]                            18.3607
Buy & Hold Return [%]                0.136376
Max. Drawdown [%]                    -2.20604
Avg. Drawdown [%]                   -0.303023
Max. Drawdown Duration       36 days 16:00:00
Avg. Drawdown Duration        4 days 06:00:00
# Trades                                   31
Win Rate [%]                          90.3226
Best Trade [%]                        2.75831
Worst Trade [%]                      -2.02095
Avg. Trade [%]                       0.548461
Max. Trade Duration          20 days 00:00:00
Avg. Trade Duration           9 days 16:00:00
Expectancy [%]                       0.776415
SQN                                   3.95528
Sharpe Ratio                      

,Equity,Exit Entry,Exit Position,Entry Price,Exit Price,P/L,Returns,Drawdown,Drawdown Duration
2019-02-11 11:00:00,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaT
2019-02-11 15:00:00,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaT
2019-02-11 19:00:00,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaT
2019-02-11 23:00:00,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaT
2019-02-12 03:00:00,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaT
2019-02-12 07:00:00,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaT
2019-02-12 11:00:00,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaT
2019-02-12 15:00:00,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaT
2019-02-12 19:00:00,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaT
2019-02-12 23:00:00,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaT
